**TODO :**
- tf-idf if not oom ?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src

/home/theo/kaggle/foursquare/src


In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# **Librairies**

In [4]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch

torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2080 Ti'

<IPython.core.display.Javascript object>

In [5]:
import gc
import cudf
import random
import warnings
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from params import DEBUG, OUT_PATH, IS_TEST, RESSOURCES_PATH
from ressources import *
from matching import *

random.seed(13)
warnings.simplefilter("ignore")
pd.options.display.max_columns = 500

<IPython.core.display.Javascript object>

## Load Data
- TODO: load precomputed probas

In [6]:
if IS_TEST:
    train = load_cleaned_data(OUT_PATH + "cleaned_data_test.csv")
    p1 = pd.read_csv(OUT_PATH + "p1_yv_test.csv")
    p2 = pd.read_csv(OUT_PATH + "p2_yv_test.csv")

#     df_p = pd.read_csv(OUT_PATH + "features_test_1.csv")

else:
    train = load_cleaned_data(OUT_PATH + "cleaned_data_train.csv")
    p1 = pd.read_csv(OUT_PATH + "p1_yv_train.csv")
    p2 = pd.read_csv(OUT_PATH + "p2_yv_train.csv")
#     df_p = pd.read_csv(OUT_PATH + "features_train_1.csv")

<IPython.core.display.Javascript object>

### Retrieve preds

In [8]:
df_p = pd.read_csv(OUT_PATH + "features_train_1.csv", usecols=["id_1", "id_2"])
# df_p = df_p.sort_values(["id_1", "id_2"]).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [1]:
skiprows = np.where(preds < THRESHOLD)

NameError: name 'np' is not defined

In [ ]:
df_p = pd.read_csv(OUT_PATH + "features_train_1.csv")
df_p = df_p.sort_values(["id_1", "id_2"]).reset_index(drop=True)

In [ ]:
DEBUG = True

In [ ]:
THRESHOLD = 0.07

In [ ]:
EXP_FOLDERS = ["../logs/lvl_1/2022-06-26/6/"]

preds = np.mean([np.load(f + "pred_oof.npy") for f in EXP_FOLDERS], 0)

In [ ]:
df_p = df_p[]

In [ ]:
preds = np.load()

In [ ]:
if DEBUG:
    p1 = p1.head(100000).copy()
    p2 = p2.head(100000).copy()
    df_p = df_p.head(100000).copy()

In [ ]:
FEATURES = list(df_p.columns[2:])

In [ ]:
train["idx"] = np.arange(len(train))

### Youri & Vincent

In [ ]:
from fe import FE2

In [ ]:
%%time
df = FE2(df_p, p1, p2, train, RESSOURCES_PATH) # call FE1

In [ ]:
df.insert(0, "id_1", p1["id"].values)
df.insert(1, "id_2", p2["id"].values)

In [ ]:
df.head()

### Théo

In [ ]:
from data.features import (
    is_equal,
    is_included,
    levenshtein,
    wratio,
    #     haversine_distance,
    #     manhattan_distance,
    #     angular_distance,
    #     euclidian_distance,
    #     angular_distance_l2,
    compute_string_distance,
    tf_idf_similarity,
)

In [ ]:
def compute_string_distances(df, string_columns, verbose=0):
    functions = [
#         gesh,
        levenshtein,
#         wratio,
    ]

    features = []
    for col in string_columns:
        df[col + "_1"].fillna("", inplace=True)
        df[col + "_2"].fillna("", inplace=True)

        df[f"{col}_len_1"] = df[col + "_1"].apply(len)
        df[f"{col}_len_2"] = df[col + "_2"].apply(len)

        df[f"{col}_len_diff"] = np.abs(df[f"{col}_len_1"] - df[f"{col}_len_2"])
        features.append(f"{col}_len_diff")

        for fct in functions:
            name = fct.__name__
            if verbose:
                print(f"- Column : {col}  -  Function : {name}")

            df[col + "_" + name] = df[[col + "_1", col + "_2"]].apply(
                lambda x: compute_string_distance(fct, x[0], x[1]), axis=1
            )
            features.append(col + "_" + name)

        # Normalize
        df[f"{col}_levenshtein"] = df[f"{col}_levenshtein"] / df[
            [f"{col}_len_1", f"{col}_len_2"]
        ].max(axis=1)

    return features


In [ ]:
def compute_position_distances(df, functions):
    features = []

    lats_1 = df["latitude_1"]
    longs_1 = df["longitude_1"]
    lats_2 = df["latitude_2"]
    longs_2 = df["longitude_2"]

    for fct in functions:
        name = fct.__name__
        df[name] = fct(lats_1, longs_1, lats_2, longs_2)
        df[name + "_s"] = fct(longs_1, lats_1, lats_2, longs_2)
        df[name + "_min"] = df[[name, name + "_s"]].min(1)

        df.drop([name, name + "_s"], axis=1, inplace=True)

        features += [name + "_min"]

    return features

In [ ]:
def angular_distance(lat1, long1, lat2, long2):
    delta_long = pd.concat([np.abs(long2 - long1), 360 - np.abs(long2 - long1)], 1).min(
        1
    )
    return np.abs(lat2 - lat1) + delta_long


def angular_distance_l2(lat1, long1, lat2, long2):
    delta_long = pd.concat([np.abs(long2 - long1), 360 - np.abs(long2 - long1)], 1).min(
        1
    )
    return np.sqrt(np.clip((lat2 - lat1) ** 2 + delta_long**2, 0, 100000))

In [ ]:
def feature_engineering_theo(df, df_p):
    features = []

    for col in ["name", "address", "url"]:
        df.loc[df[col] == "", col] = np.nan

    #     print('- Computing nan features')
    #     features += compute_nan_features(df_p, NAN_COLS)

    #     for name, folder in NN_FT_FOLDERS:
    #         print(f'- Adding features using model {name}')
    #         nn_preds = np.load(OUT_PATH + f"fts_{name}.npy").astype(np.float16)
    #         nn_preds = torch.from_numpy(nn_preds).cuda()

    #         features += compute_nn_distances(df_p, nn_preds, suffix="_" + name)

    #         del nn_preds
    #         gc.collect()
    #         torch.cuda.empty_cache()

    print("- Computing position distances")
    DIST_FCTS = [angular_distance, angular_distance_l2]
    features += compute_position_distances(df_p, DIST_FCTS)

    #     TF_IDF_COLS = ["name", "address", "url"]
    #     TF_IDF_PARAMS = [
    #         ((3, 3), "char_wb"),  # char trigrams
    #     ]
    #     for col in TF_IDF_COLS:
    #         for ngram_range, analyzer in TF_IDF_PARAMS:
    #             ft_name = f"{col}_tf_idf_{ngram_range[0]}{ngram_range[1]}_{analyzer}_sim"
    #             print(f"- Computing feature {ft_name}")

    #             tf_idf = TfidfVectorizer(
    #                 use_idf=False, ngram_range=ngram_range, analyzer=analyzer
    #             )
    #             tf_idf_mat = tf_idf.fit_transform(cudf.from_pandas(df[col].fillna("nan")))

    #             df_p[ft_name] = tf_idf_similarity(df_p, tf_idf_mat)
    #             features.append(ft_name)

    if not isinstance(df_p, pd.DataFrame):
        df_p = df_p.to_pandas()

    FEATURES_SAME = [
        ("state", is_equal),
        ("zip", is_included),
        ("city", is_included),
    ]

    for col, fct in FEATURES_SAME:
        print(f"- Computing feature same_{col}")
        df_p[f"same_{col}"] = (
            df_p[[f"{col}_1", f"{col}_2"]]
            .fillna("")
            .apply(lambda x: fct(x[0], x[1]), axis=1)
            .astype(float)  # parallel_apply
        )

        features.append(f"same_{col}")

    STRING_DIST_COLS = ["name", "address", "url"]
    features += compute_string_distances(df_p, STRING_DIST_COLS, verbose=1)

    to_keep = ["id_1", "id_2"] + features
    df_p.drop([c for c in df_p.columns if c not in to_keep], axis=1, inplace=True)

    return df_p, features

In [ ]:
import cudf
from cuml.feature_extraction.text import TfidfVectorizer

In [ ]:
cols = [
    "id",
    "name",
    "latitude",
    "longitude",
    "address",
    "country",
    "url",
    "phone",
    "city",
    "state",
    "zip",
    "categories",
    "idx",
]
pairs = pd.concat([p1[cols], p2[cols]], axis=1)
pairs.columns = [c + "_1" for c in cols] + [c + "_2" for c in cols]

In [ ]:
%%time

df_theo, fts_theo = feature_engineering_theo(train.copy(), pairs)

## Merge & Save

In [ ]:
df_merged = df.merge(df_theo, on=["id_1", "id_2"])

In [ ]:
df_merged.head()

In [ ]:
# import matplotlib.pyplot as plt

# plt.scatter(df_merged["angular_distance_min"], df_merged["dist_r1"])

In [ ]:
corrs = df_merged.corr()
for col in corrs.columns:
    print(col, corrs.loc[corrs[col] > 0.9, col])
    print("")

In [ ]:
if not DEBUG:
    if IS_TEST:
        df_merged.to_csv(OUT_PATH + "features_test_1.csv", index=False)
    else:
        df_merged.to_csv(OUT_PATH + "features_train_1.csv", index=False)